In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\project\\Text Summerization'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    learning_rate: float
    fp16: bool
    save_total_limit: int

In [5]:
from CutYourText.constants import *
from CutYourText.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            learning_rate = params.learning_rate,
            fp16 = params.fp16,
            save_total_limit = params.save_total_limit
        )

        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch
from torch.optim import AdamW

e:\project\Text Summerization\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-18 06:33:15,589: INFO: config: PyTorch version 2.4.0+cu121 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_bart = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bart)
        
        # Load data
        dataset_dialogsum_pt = load_from_disk(self.config.data_path)

        # Define training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.eval_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps= self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            learning_rate=self.config.learning_rate,
            fp16=self.config.fp16,
            save_total_limit=self.config.save_total_limit
        ) 

        # Define the optimizer using torch.optim.AdamW
        optimizer = AdamW(
            model_bart.parameters(),
            lr=3e-5,             # Fine-tuning learning rate for text summarization
            weight_decay=0.01,    # Standard weight decay
            eps=1e-8              # Epsilon to avoid division by zero in Adam
        )

        # Initialize Trainer with the custom optimizer
        trainer = Trainer(
            model=model_bart, 
            args=trainer_args,
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_dialogsum_pt["train"], 
            eval_dataset=dataset_dialogsum_pt["validation"],
            optimizers=(optimizer, None)  
        )
        
        trainer.train()

        # Save model
        model_bart.save_pretrained(os.path.join(self.config.root_dir, "bart-dialogsum-model"))
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-08-18 06:33:52,528: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-18 06:33:52,540: INFO: common: yaml file: params.yaml loaded successfully]


[2024-08-18 06:33:52,541: INFO: common: created directory at: artifacts]
[2024-08-18 06:33:52,543: INFO: common: created directory at: artifacts/model_trainer]


e:\project\Text Summerization\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
e:\project\Text Summerization\venv\lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 0/2334 [00:00<?, ?it/s]e:\project\Text Summerization\venv\lib\site-packages\transformers\models\bart\modeling_bart.py:496: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_uti

{'loss': 1.8231, 'grad_norm': 5.613476276397705, 'learning_rate': 2.88e-06, 'epoch': 0.06}


  4%|▍         | 100/2334 [26:18<10:46:40, 17.37s/it]

...

{'loss': 0.6315, 'grad_norm': 3.1077840328216553, 'learning_rate': 1.4231188658669574e-06, 'epoch': 2.89}


 99%|█████████▊| 2300/2334 [11:22:56<10:42, 18.90s/it]

{'loss': 0.6307, 'grad_norm': 3.819241762161255, 'learning_rate': 6.052344601962923e-07, 'epoch': 2.95}


100%|██████████| 2334/2334 [11:33:39<00:00, 18.89s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
100%|██████████| 2334/2334 [11:33:50<00:00, 17.84s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, '

{'train_runtime': 41630.2394, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.056, 'train_loss': 0.8786970373903877, 'epoch': 3.0}
